In [1]:
# =========================
# chargement de donnees
# =========================
import os
from zipfile import ZipFile
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from google.colab import drive

# 1) Monter Google Drive
drive.mount('/content/drive')

# 2) Chemin du zip
zip_path = "/content/drive/MyDrive/Colab Notebooks/COVID-19_Radiography_dataset.zip"
extract_path = "/content/COVID-19_Radiography_Dataset"

# 3) Dézipper si besoin
if not os.path.exists(extract_path):
    with ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("/content/")



Mounted at /content/drive


In [2]:
# =========================
# Réorganisation (2 classes)
# =========================
import shutil

prepared_data_dir = "/content/covid_data_prepared"
os.makedirs(prepared_data_dir, exist_ok=True)

# On ne garde que ces 2 classes
classes = ["COVID", "Normal"]

def find_src_dir(cls):
    
    candidates = [
        os.path.join("/content/COVID-19_Radiography_Dataset", cls, "images"),
        os.path.join("/content/COVID-19_Radiography_Dataset", cls),
        os.path.join("/content", cls, "images"),
        os.path.join("/content", cls),
    ]
    for p in candidates:
        if os.path.isdir(p):
            return p
    raise FileNotFoundError(f"Dossier images introuvable pour la classe '{cls}'.")

copied_counts = {}
for cls in classes:
    src_img_dir = find_src_dir(cls)
    dst_class_dir = os.path.join(prepared_data_dir, cls.replace(" ", "_"))
    os.makedirs(dst_class_dir, exist_ok=True)

    # Copie toutes les images trouvées (filtre simples extensions)
    exts = {".png", ".jpg", ".jpeg", ".bmp"}
    n = 0
    for filename in os.listdir(src_img_dir):
        if os.path.splitext(filename.lower())[1] in exts:
            shutil.copy(os.path.join(src_img_dir, filename),
                        os.path.join(dst_class_dir, filename))
            n += 1
    copied_counts[cls] = n

print("✅ Réorganisation terminée. Structure ImageFolder prête.")
print("Comptes copiés:", copied_counts)

✅ Réorganisation terminée. Structure ImageFolder prête.
Comptes copiés: {'COVID': 3616, 'Normal': 10192}


In [9]:
# =========================
# Paramètres globaux, Seeds & Device
# =========================
import os, random, numpy as np, torch

# -- chemins --
prepared_data_dir = "/content/covid_data_prepared"

# -- hyper généraux --
SEED = 42
IMG_SIZE = 224
BATCH_TRAIN = 32
BATCH_EVAL  = 128
WORKERS = 2   

# -- reproductibilité & perf --
def set_seeds(seed=SEED):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed)

set_seeds()

DEVICE = torch.device("cpu")
print("Device:", DEVICE)


Device: cpu


In [10]:
# =========================
# Bloc 4 — Réduction équilibrée + splits (identique baseline)
# =========================


PER_CLASS = 500      
VAL_FRAC  = 0.10     
TEST_FRAC = 0.20      
rng = np.random.RandomState(SEED)

base = datasets.ImageFolder(prepared_data_dir)
print("Classes :", base.classes)
print("Mapping :", base.class_to_idx) 

POS_NAME = "COVID"
POS_IDX  = base.class_to_idx[POS_NAME]   
print("Index classe positive (COVID) =", POS_IDX)



name_to_idx = base.class_to_idx
KEEP_NAMES = ["COVID", "Normal"]
KEEP_IDX = [name_to_idx[n] for n in KEEP_NAMES]


idxs_by_class = {ci: [] for ci in KEEP_IDX}
for i, (_, y) in enumerate(base.samples):
    if y in KEEP_IDX:
        idxs_by_class[y].append(i)

# réduction équilibrée
kept_by_class = {}
for c, idxs in idxs_by_class.items():
    idxs = np.array(idxs); rng.shuffle(idxs)
    k = min(PER_CLASS, len(idxs))
    kept_by_class[c] = idxs[:k]

print("Après réduction :",
      {base.classes[c]: len(kept_by_class[c]) for c in KEEP_IDX},
      "Total:", sum(len(v) for v in kept_by_class.values()))

# split stratifié sur les indices RÉDUITS
train_idx, val_idx, test_idx = [], [], []
for c in KEEP_IDX:
    idxs = kept_by_class[c].copy(); rng.shuffle(idxs)
    n = len(idxs)
    n_test = int(round(n * TEST_FRAC))
    n_val  = int(round((n - n_test) * VAL_FRAC))
    test_idx.extend(idxs[:n_test].tolist())
    val_idx.extend(idxs[n_test:n_test+n_val].tolist())
    train_idx.extend(idxs[n_test+n_val:].tolist())

print(f"Splits -> train:{len(train_idx)} | val:{len(val_idx)} | test:{len(test_idx)}")



Classes : ['COVID', 'Normal']
Mapping : {'COVID': 0, 'Normal': 1}
Index classe positive (COVID) = 0
Après réduction : {'COVID': 500, 'Normal': 500} Total: 1000
Splits -> train:720 | val:80 | test:200


In [11]:
# =========================
# Bloc 5 — DataLoaders (identique baseline)
# =========================
from torch.utils.data import Subset, DataLoader

MEAN = [0.485, 0.456, 0.406]
STD  = [0.229, 0.224, 0.225]

train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD),
])
eval_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD),
])

attack_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

base_train  = datasets.ImageFolder(prepared_data_dir, transform=train_tfms)
base_eval   = datasets.ImageFolder(prepared_data_dir, transform=eval_tfms)
base_attack = datasets.ImageFolder(prepared_data_dir, transform=attack_tfms)

train_ds       = Subset(base_train,  train_idx)
val_ds         = Subset(base_eval,   val_idx)
test_ds        = Subset(base_eval,   test_idx)
attack_test_ds = Subset(base_attack, test_idx)

loader_kwargs = dict(pin_memory=(DEVICE.type=="cuda"))
if WORKERS > 0:
    loader_kwargs.update(num_workers=WORKERS, persistent_workers=True, prefetch_factor=2)

train_loader       = DataLoader(train_ds,       batch_size=BATCH_TRAIN, shuffle=True,  **loader_kwargs)
val_loader         = DataLoader(val_ds,         batch_size=BATCH_EVAL,  shuffle=False, **loader_kwargs)
test_loader        = DataLoader(test_ds,        batch_size=BATCH_EVAL,  shuffle=False, **loader_kwargs)
attack_test_loader = DataLoader(attack_test_ds, batch_size=BATCH_EVAL,  shuffle=False, **loader_kwargs)

print("✅ DataLoaders prêts.")


✅ DataLoaders prêts.


In [12]:
# =========================
# Bloc 6 — Modèle CNN (identique baseline)
# =========================
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2, dropout=0.3):
        super().__init__()
        self.b1 = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1, bias=False),
            nn.BatchNorm2d(32), nn.ReLU(inplace=True), nn.MaxPool2d(2)
        )
        self.b2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64), nn.ReLU(inplace=True), nn.MaxPool2d(2)
        )
        self.b3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1, bias=False),
            nn.BatchNorm2d(128), nn.ReLU(inplace=True), nn.MaxPool2d(2)
        )
        self.gap  = nn.AdaptiveAvgPool2d((1,1))
        self.drop = nn.Dropout(dropout)
        self.fc   = nn.Linear(128, num_classes)
        # init propre
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, nonlinearity="relu")
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight); nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight); nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.b1(x); x = self.b2(x); x = self.b3(x)
        x = self.gap(x); x = torch.flatten(x, 1)
        x = self.drop(x); x = self.fc(x)
        return x

    def extract_features(self, x):
        x = self.b1(x); x = self.b2(x); x = self.b3(x)
        x = self.gap(x); return torch.flatten(x, 1)

model = SimpleCNN(num_classes=2, dropout=0.3).to(DEVICE)




In [13]:
!pip -q install adversarial-robustness-toolbox==1.17.1


In [14]:
# =========================
# Bloc 7 — Mixed Adversarial Training 
# =========================
import time, copy, numpy as np, torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# --- Optim / critères / early stop
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
EPOCHS, PATIENCE = 25, 5
best_val, no_impr = float("inf"), 0
best_state = copy.deepcopy(model.state_dict())

# --- Espace NORMALISÉ 
MEAN_T = torch.tensor(MEAN, device=DEVICE).view(1, 3, 1, 1)
STD_T  = torch.tensor(STD,  device=DEVICE).view(1, 3, 1, 1)
X_MIN  = (0 - MEAN_T) / STD_T
X_MAX  = (1 - MEAN_T) / STD_T

# --- Hyperparams MAT
EPS_TRAIN   = 4/255
LAMBDA_ADV  = 0.3
RHO_ADV     = 0.5          
rng = np.random.RandomState(42)

@torch.no_grad()
def clamp_normed(x):
    return torch.max(torch.min(x, X_MAX), X_MIN)

def fgsm_norm(x, y, eps=EPS_TRAIN):
    model.eval()
    x_adv = x.detach().clone().requires_grad_(True)
    logits = model(x_adv)
    loss = criterion(logits, y)
    model.zero_grad(set_to_none=True)
    loss.backward()
    step = (eps / STD_T).to(x.device)
    x_adv = x_adv + step * x_adv.grad.detach().sign()
    x_adv = clamp_normed(x_adv).detach()
    model.train()
    return x_adv

def pgd_norm(x, y, eps=EPS_TRAIN, alpha=None, iters=3, random_start=True):
    if alpha is None: alpha = eps / 4
    model.eval()
    x0 = x.detach()
    if random_start:
        delta0 = torch.empty_like(x0).uniform_(-eps, eps) / STD_T
        x_adv = clamp_normed(x0 + delta0)
    else:
        x_adv = x0.clone()
    for _ in range(iters):
        x_adv.requires_grad_(True)
        logits = model(x_adv)
        loss = criterion(logits, y)
        model.zero_grad(set_to_none=True)
        loss.backward()
        step = (alpha / STD_T).to(x.device)
        x_adv = x_adv + step * x_adv.grad.detach().sign()
        delta = torch.max(torch.min(x_adv - x0, (eps / STD_T)), -(eps / STD_T))
        x_adv = clamp_normed(x0 + delta).detach()
    model.train()
    return x_adv

def bim_norm(x, y, eps=EPS_TRAIN, alpha=None, iters=5):
    if alpha is None: alpha = eps / 10
    model.eval()
    x0 = x.detach()
    x_adv = x0.clone()
    for _ in range(iters):
        x_adv.requires_grad_(True)
        logits = model(x_adv)
        loss = criterion(logits, y)
        model.zero_grad(set_to_none=True)
        loss.backward()
        step = (alpha / STD_T).to(x.device)
        x_adv = x_adv + step * x_adv.grad.detach().sign()
        delta = torch.max(torch.min(x_adv - x0, (eps / STD_T)), -(eps / STD_T))
        x_adv = clamp_normed(x0 + delta).detach()
    model.train()
    return x_adv

def evaluate(model, loader):
    model.eval(); loss_sum = 0.0; n = 0; correct = 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(DEVICE); y = y.to(DEVICE)
            logits = model(x); loss = criterion(logits, y)
            loss_sum += loss.item() * y.size(0); n += y.size(0)
            correct += (logits.argmax(1) == y).sum().item()
    return loss_sum / n, correct / n

# --- Entraînement MAT
for epoch in range(1, EPOCHS + 1):
    model.train(); t0 = time.time(); run_loss = 0.0; n = 0
    for i, (x, y) in enumerate(train_loader):
        x = x.to(DEVICE); y = y.to(DEVICE)

        a = ("fgsm", "pgd", "bim")[i % 3]

        B = x.size(0)
        n_adv = max(1, int(RHO_ADV * B))
        idx = rng.permutation(B)[:n_adv]
        x_sub, y_sub = x[idx], y[idx]

        # génération des adversaires pour la sous-partie
        if a == "fgsm":
            x_adv_sub = fgsm_norm(x_sub, y_sub, eps=EPS_TRAIN)
        elif a == "pgd":
            x_adv_sub = pgd_norm(x_sub, y_sub, eps=EPS_TRAIN, alpha=EPS_TRAIN/4, iters=3, random_start=True)
        else:
            x_adv_sub = bim_norm(x_sub, y_sub, eps=EPS_TRAIN, alpha=EPS_TRAIN/10, iters=5)

        optimizer.zero_grad(set_to_none=True)

        
        logits_clean = model(x)
        loss_clean   = criterion(logits_clean, y)

        logits_adv   = model(x_adv_sub)
        loss_adv     = criterion(logits_adv, y_sub)

        loss = (1.0 - LAMBDA_ADV) * loss_clean + LAMBDA_ADV * loss_adv
        loss.backward(); optimizer.step()

        run_loss += loss.item() * y.size(0); n += y.size(0)

    train_loss = run_loss / n
    val_loss, val_acc = evaluate(model, val_loader)
    print(f"[MAT-FPB] Epoch {epoch:02d} | train_loss={train_loss:.4f} | val_loss={val_loss:.4f} | val_acc={val_acc:.3f} | {time.time()-t0:.1f}s")

    if val_loss < best_val - 1e-4:
        best_val = val_loss; no_impr = 0
        best_state = copy.deepcopy(model.state_dict())
    else:
        no_impr += 1
        if no_impr >= PATIENCE:
            print("Early stopping."); break

# --- Évaluation clean (sur test_loader normalisé)
model.load_state_dict(best_state); model.eval()
y_true, y_pred, y_prob = [], [], []
with torch.no_grad():
    for x, y in test_loader:
        x = x.to(DEVICE)
        logits = model(x)
        prob_pos = torch.softmax(logits, dim=1)[:, POS_IDX].cpu().numpy() 
        y_prob.extend(prob_pos)
        y_pred.extend(logits.argmax(1).cpu().numpy())
        y_true.extend(y.numpy())

y_true_pos = [1 if t == POS_IDX else 0 for t in y_true]

print("DEF(MAT-FPB) Test | acc:{:.4f} | prec:{:.4f} | rec:{:.4f} | f1:{:.4f} | auc:{:.4f}".format(
    accuracy_score(y_true, y_pred),
    precision_score(y_true, y_pred, pos_label=POS_IDX, zero_division=0),
    recall_score(y_true, y_pred,    pos_label=POS_IDX, zero_division=0),
    f1_score(y_true, y_pred,        pos_label=POS_IDX, zero_division=0),
    roc_auc_score(y_true_pos, y_prob),
))


[MAT-FPB] Epoch 01 | train_loss=0.7374 | val_loss=0.7033 | val_acc=0.662 | 320.1s
[MAT-FPB] Epoch 02 | train_loss=0.6766 | val_loss=0.7572 | val_acc=0.637 | 272.1s
[MAT-FPB] Epoch 03 | train_loss=0.6552 | val_loss=0.6600 | val_acc=0.637 | 268.5s
[MAT-FPB] Epoch 04 | train_loss=0.6475 | val_loss=0.7064 | val_acc=0.650 | 264.0s
[MAT-FPB] Epoch 05 | train_loss=0.6489 | val_loss=0.6982 | val_acc=0.700 | 265.9s
[MAT-FPB] Epoch 06 | train_loss=0.6420 | val_loss=0.6326 | val_acc=0.662 | 265.8s
[MAT-FPB] Epoch 07 | train_loss=0.6413 | val_loss=0.6112 | val_acc=0.700 | 264.9s
[MAT-FPB] Epoch 08 | train_loss=0.6312 | val_loss=0.6253 | val_acc=0.675 | 266.8s
[MAT-FPB] Epoch 09 | train_loss=0.6017 | val_loss=0.6180 | val_acc=0.675 | 267.6s
[MAT-FPB] Epoch 10 | train_loss=0.6212 | val_loss=0.6006 | val_acc=0.725 | 268.7s
[MAT-FPB] Epoch 11 | train_loss=0.5985 | val_loss=0.5859 | val_acc=0.688 | 264.6s
[MAT-FPB] Epoch 12 | train_loss=0.5942 | val_loss=0.5858 | val_acc=0.675 | 262.5s
[MAT-FPB] Epoch 

In [15]:
# =========================
# Bloc 8 — Préparation attaques (identique baseline, CPU-only)
# =========================

import numpy as np, torch

def dataloader_to_numpy(dl):
    xs, ys = [], []
    for x, y in dl:
        xs.append(x.numpy()); ys.append(y.numpy())
    return np.concatenate(xs, 0), np.concatenate(ys, 0)

X_test_np, y_test_np = dataloader_to_numpy(attack_test_loader)

# Tenseurs de normalisation (pour évaluer proprement après génération adv)
MEAN_T = torch.tensor([0.485, 0.456, 0.406], device=DEVICE).view(1, 3, 1, 1)
STD_T  = torch.tensor([0.229, 0.224, 0.225], device=DEVICE).view(1, 3, 1, 1)

def eval_np_batchwise(X_np, y_np, batch=BATCH_EVAL, pos_idx=POS_IDX):
   urne acc, precision, recall, f1, auc.
    
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
    y_true, y_pred, y_prob = [], [], []
    model.eval()
    with torch.no_grad():
        for i in range(0, len(X_np), batch):
            xt = torch.from_numpy(X_np[i:i+batch]).to(DEVICE).float()  
            xt = (xt - MEAN_T) / STD_T                                 
            logits = model(xt)
            prob_pos = torch.softmax(logits, dim=1)[:, pos_idx].cpu().numpy()
            pred     = logits.argmax(1).cpu().numpy()
            y_prob.extend(prob_pos)
            y_pred.extend(pred)
            y_true.extend(y_np[i:i+batch])

    
    y_true_pos = [1 if t == pos_idx else 0 for t in y_true]

    return dict(
        acc=accuracy_score(y_true, y_pred),
        precision=precision_score(y_true, y_pred, pos_label=pos_idx, zero_division=0),
        recall=recall_score(y_true, y_pred,    pos_label=pos_idx, zero_division=0),
        f1=f1_score(y_true, y_pred,            pos_label=pos_idx, zero_division=0),
        auc=roc_auc_score(y_true_pos, y_prob),
    )


In [16]:
# =========================
# Bloc 9 — Attaques FGSM / PGD / BIM 
# =========================
import numpy as np
import torch
import torch.nn as nn
from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent, BasicIterativeMethod
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 1) Classifier ART 
clf = PyTorchClassifier(
    model=model,
    loss=nn.CrossEntropyLoss(),
    optimizer=torch.optim.Adam(model.parameters(), lr=1e-3),
    input_shape=(3, IMG_SIZE, IMG_SIZE),
    nb_classes=2,
    clip_values=(0.0, 1.0),
    preprocessing=(np.array(MEAN, dtype=np.float32), np.array(STD, dtype=np.float32)),
    device_type="cpu", 
)

# 2) Attaques en streaming
PS_GRID = [2/255, 4/255, 8/255]
results_fpb = []

def eval_adv_stream(attack, X_np, y_np, batch=BATCH_EVAL, pos_idx=POS_IDX):
    """
    Génère X_adv en [0,1] avec ART, applique la normalisation PyTorch,
    puis évalue le modèle. pos_idx = index de la classe positive (COVID).
    """
    y_true, y_pred, y_prob = [], [], []
    model.eval()
    for i in range(0, len(X_np), batch):
        Xb = X_np[i:i+batch]; yb = y_np[i:i+batch]
        Xab = attack.generate(Xb)  
        with torch.no_grad():
            xt = torch.from_numpy(Xab).to(DEVICE).float()  
            xt = (xt - MEAN_T) / STD_T                    
            logits   = model(xt)
            prob_pos = torch.softmax(logits, dim=1)[:, pos_idx].cpu().numpy()
            pred     = logits.argmax(1).cpu().numpy()
        y_true.extend(yb); y_pred.extend(pred); y_prob.extend(prob_pos)

    
    y_true_pos = [1 if t == pos_idx else 0 for t in y_true]

    return dict(
        acc      = accuracy_score(y_true, y_pred),
        precision= precision_score(y_true, y_pred, pos_label=pos_idx, zero_division=0),
        recall   = recall_score(y_true, y_pred,    pos_label=pos_idx, zero_division=0),
        f1       = f1_score(y_true, y_pred,        pos_label=pos_idx, zero_division=0),
        auc      = roc_auc_score(y_true_pos, y_prob),
    )

# FGSM
for eps in PS_GRID:
    atk = FastGradientMethod(estimator=clf, eps=eps, batch_size=BATCH_EVAL)
    m = eval_adv_stream(atk, X_test_np, y_test_np, batch=BATCH_EVAL)
    results_fpb.append(dict(attack="FGSM", eps=eps, step=None, iters=1, **m))
    print(f"FGSM eps={eps:.5f} | acc={m['acc']:.4f} prec={m['precision']:.4f} "
          f"rec={m['recall']:.4f} f1={m['f1']:.4f} auc={m['auc']:.4f}")

# PGD 
for eps in PS_GRID:
    atk = ProjectedGradientDescent(
        estimator=clf, eps=eps, eps_step=eps/8, max_iter=20,
        batch_size=BATCH_EVAL, targeted=False, num_random_init=0
    )
    m = eval_adv_stream(atk, X_test_np, y_test_np, batch=BATCH_EVAL)
    results_fpb.append(dict(attack="PGD", eps=eps, step=eps/8, iters=20, random_init=False, **m))
    print(f"PGD  eps={eps:.5f} | acc={m['acc']:.4f} prec={m['precision']:.4f} "
          f"rec={m['recall']:.4f} f1={m['f1']:.4f} auc={m['auc']:.4f}")

# BIM
for eps in PS_GRID:
    atk = BasicIterativeMethod(estimator=clf, eps=eps, eps_step=eps/12, max_iter=12, batch_size=BATCH_EVAL)
    m = eval_adv_stream(atk, X_test_np, y_test_np, batch=BATCH_EVAL)
    results_fpb.append(dict(attack="BIM", eps=eps, step=eps/12, iters=12, **m))
    print(f"BIM  eps={eps:.5f} | acc={m['acc']:.4f} prec={m['precision']:.4f} "
          f"rec={m['recall']:.4f} f1={m['f1']:.4f} auc={m['auc']:.4f}")


FGSM eps=0.00784 | acc=0.6700 prec=0.6466 rec=0.7500 f1=0.6944 auc=0.7483
FGSM eps=0.01569 | acc=0.6600 prec=0.6633 rec=0.6500 f1=0.6566 auc=0.7001
FGSM eps=0.03137 | acc=0.5550 prec=0.5679 rec=0.4600 f1=0.5083 auc=0.5625


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD  eps=0.00784 | acc=0.6700 prec=0.6518 rec=0.7300 f1=0.6887 auc=0.7387


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

BIM  eps=0.01569 | acc=0.6200 prec=0.6224 rec=0.6100 f1=0.6162 auc=0.6600


PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

BIM  eps=0.03137 | acc=0.4300 prec=0.4146 rec=0.3400 f1=0.3736 auc=0.4240


In [ ]:
from art.attacks.evasion import CarliniL2Method

# =========================
# Carlini & Wagner L2 
# =========================
CONF = 0
BATCH_ATTACK = 32  


atk = CarliniL2Method(
        classifier=clf,
        targeted=False,
        confidence=CONF,
        max_iter=75,
        binary_search_steps=1,
        learning_rate=0.02,
        initial_const=0.3,
        batch_size=BATCH_ATTACK
        )

m = eval_adv_stream(atk, X_test_np, y_test_np, batch=BATCH_ATTACK, pos_idx=POS_IDX)

results_cw = [dict(
    attack="CW-L2",
    confidence=CONF,
    max_iter=75,
    bsearch=1,
    c0=0.3,
    **m
)]

print(
    f"C&W conf={CONF} | acc={m['acc']:.4f} | prec={m['precision']:.4f} "
    f"| rec={m['recall']:.4f} | f1={m['f1']:.4f} | auc={m['auc']:.4f}"
)


C&W L_2:   0%|          | 0/1 [00:00<?, ?it/s]